In [1]:
import tensorflow as tf
import numpy as np
mnist = tf.keras.datasets.mnist

In [2]:
class layer():
    def __init__(self,inputs,units,activation='relu',use_bias=True,kernel_initializer=None,bias_regularizer=False,dropout=False):
        self.inputs = inputs
        self.units = units
        self.activation = activation
        self.use_bias = use_bias
#         self.kernel_regularizer=kernel_regularizer
        self.bias_regularizer=bias_regularizer
        self.dropout = dropout
        self.b = np.zeros((self.units,1))
        self.inward = 0
        self.z = 0
        self.a = 0
        self.dw = 0
        self.db = 0
        self.da = 0
        self.initialize_weight()
        
    def sigmoid(self, x):
        #applying the sigmoid function
        return 1 / (1 + np.exp(-x))

    def initialize_weight(self):
        if(self.activation == 'relu'):
            self.w = np.random.rand(self.units,self.inputs) * np.sqrt(2.0/self.inputs)
        elif(self.activation == 'tanh'):
            self.w = np.random.rand(self.units,self.inputs) * np.sqrt(1.0/self.inputs)
        else:
            self.w = np.random.rand(self.units,self.inputs) * np.sqrt(2.0/(self.units+self.inputs))
            
    def sigmoid_d(self, x):
        return x * (1 - x)
    
    def softmax(self,x):
        x = x - np.mean(x)
#         print x
        expx = np.exp(x)
        return expx / expx.sum()
    
    def tanh(self, x):
        #applying the tanh function
        return 1 / (1 + np.exp(-x))

    def tanh_d(self, x):
        return (1 + x) * (1 - x)
    
    def relu(self,x):
        return np.maximum(0,x) 
    
    def relu_d(self,x):
        x[x<=0] = 0
        x[x>0] = 1
        return x

    def update(self,alpha):
#         print "W = ",self.w
#         print "b = ",self.b
#         print "dW = ",self.dw
#         print "db = ",self.db
        self.w += alpha * self.dw
        self.b += alpha * self.db
        
    
    def forward(self,inward):
        inward = inward.reshape((inward.shape[0],1))
        self.inward = inward
        self.z = np.dot(self.w,inward) 
        self.a = self.z
        if(self.activation == 'sigmoid'):
            self.a = self.sigmoid(self.a)
        elif(self.activation == 'tanh'):
            self.a = self.tanh(self.a)
        elif(self.activation == 'relu'):
            self.a = self.relu(self.a)
        elif(self.activation == 'softmax'):
            self.a = self.softmax(self.a)
        else:
            pass
        return self.a
            
    def backward(self,a):
#         print a.shape
        if(self.activation == 'sigmoid'):
            self.dz = np.multiply(a,self.sigmoid(self.z))
        elif(self.activation == 'tanh'):
            self.dz = np.multiply(a,self.tanh(self.z))
        elif(self.activation == 'relu'):
            self.dz = np.multiply(a,self.relu(self.z))
        elif(self.activation == 'softmax'):
            self.dz = a
        else:
            self.dz = 0 * a
            
        self.dw = np.dot(self.dz, self.inward.T)
        self.db = np.sum(self.dz, axis=1, keepdims=True)
        next_a = np.dot(self.w.T, self.dz)
        return next_a

In [3]:
class model():
    def __init__(self,data,data_label,number_of_class,alpha=0.1,accuracy=0.9):
        self.layers = []
        self.alpha = alpha
        self.accuracy = accuracy
        self.data = data
        self.data_label = data_label 
        self.number_of_class = number_of_class
        self.cm = np.zeros((self.number_of_class,self.number_of_class),dtype=int)
        self.eplison = 10**(-4)
        
    def add(self,layer):
        self.layers.append(layer)
    
    def train(self):
        i = 0
        while(i < 10):
            print i
            self.cm = np.zeros((self.number_of_class,self.number_of_class),dtype=int)
            self.do_epoch()
            self.get_accuracy()
            i+=1
#             print self.layers[0].dw

    def get_accuracy(self):
        print self.cm
        print np.trace(self.cm)/(1.0*np.sum(self.cm))

    def update(self):
        for layer in self.layers:
            layer.update(self.alpha)
    
    def get_cost(self,y,y_label):
        
#         print y_label,y
        if(self.number_of_class > 2):
            y_label = y_label.reshape((y_label.shape[0],1))
            return (y_label - y)
        else:
            if (y_label == 1):
                return -1*np.log(y + self.eplison)
            else:
                return -1*np.log(1 - y + self.eplison)
    
    def predict(self,sample):
        for layer in self.layers:
            sample = layer.forward(sample)
        return sample.argmax()
    
    def do_epoch(self):
        total_cost = 0
        for j in range(len(self.data)):
            y_label = self.data_label[j]
            x = self.data[j]
            y = self.forprop(x)
            loss = self.get_cost(y,y_label)
#             print y_label,y
#             print y_label.argmax(),y.argmax()
#             print self.number_of_class
            if(self.number_of_class > 2):
                self.cm[y_label.argmax()][y.argmax()] += 1
#                 print self.cm
            else:
                if(y>0.5):
                    self.cm[y_label][1] += 1
                else:
                    self.cm[y_label][0] += 1
            self.backprop(loss)
            self.update()
#             total_cost += loss
            
    def forprop(self,sample):
        for layer in self.layers:
#             print sample
            sample = layer.forward(sample)
                  
        return sample       
    
    def backprop(self,loss):
        sample = loss
        for i in range(len(self.layers)-1,-1,-1):
            sample = self.layers[i].backward(sample)
        

In [4]:
(x_train, y_train_label),(x_test, y_test_label) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
x_train = x_train.reshape([60000,784])
x_test = x_test.reshape([10000,784])
y_train = np.zeros((y_train_label.shape[0], 10))
y_train[np.arange(y_train_label.shape[0]), y_train_label] = 1
y_test = np.zeros((y_test_label.shape[0], 10))
y_test[np.arange(y_test_label.shape[0]), y_test_label] = 1

In [6]:
nn = model(x_train,y_train,10,alpha=0.001)
layer1 = layer(784,196)
layer2 = layer(196,58)
layer3 = layer(58,10,activation='softmax')
nn.add(layer1)
nn.add(layer2)
nn.add(layer3)
nn.train()

0
[[5497    1   85   40   19   79   70   39   76   17]
 [  38 6045  177  131   36   34   66   23  173   19]
 [ 102   71 5030  128  120   36  179   94  160   38]
 [ 104   49  204 5105   19  250   47   79  217   57]
 [  26   25  120   24 4991   17  137   44   64  394]
 [ 213   37   90  349   65 4057  151   63  316   80]
 [ 103   34  181   30  101  104 5269   13   78    5]
 [  76   49  120   55  102   35   27 5480   75  246]
 [  77  149  202  232   48  192   82   35 4725  109]
 [  69   27   89  108  403   73   32  294  134 4720]]
0.84865
1
[[5757    0   17    7    6   15   49   19   34   19]
 [   1 6585   36   33   10    3    7   16   42    9]
 [  40   29 5559   61   43   10   48   61   92   15]
 [  11   21  103 5680    4  120    8   42  110   32]
 [   7   14   19    3 5486    2   75   18   19  199]
 [  54   14   24  129   24 4949   88   13   74   52]
 [  43   14   19    4   28   57 5725    0   27    1]
 [  20   31   60   25   49   11    2 5968   15   84]
 [  19   64   64  105   27   68  

In [7]:
# class1 = np.loadtxt('NLS/Class1.txt')
# class2 = np.loadtxt('NLS/Class2.txt')
# alldata = []
# for i in class1:
#     alldata.append((i,0))
# for i in class2:
#     alldata.append((i,1))
# np.random.shuffle(alldata)
# train = alldata[:int(len(alldata)*0.75)]
# test = alldata[int(len(alldata)*0.75):]
# x_train = []
# x_test = []
# y_train = []
# y_test = []

# for i in train:
#     x_train.append(i[0])
#     y_train.append(i[1])

# for i in test:
#     x_test.append(i[0])
#     y_test.append(i[1])

# x_train = np.asarray(x_train)
# y_train = np.asarray(y_train)
# x_train = np.asarray(x_test)
# y_test = np.asarray(y_test)


In [8]:
# nn = model(x_train,y_train,2)
# layer1 = layer(2,8)
# layer2 = layer(8,6)
# layer3 = layer(6,2)
# layer4 = layer(2,1,activation='sigmoid')
# nn.add(layer1)
# nn.add(layer2)
# nn.add(layer3)
# nn.add(layer4)
# nn.train()